## Modeling

### Imports

In [1]:
# Some basic imports
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from keras.callbacks.callbacks import EarlyStopping
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.losses import categorical_crossentropy
from keras.optimizers import Adam
from keras.regularizers import l2
from keras.models import load_model

Using TensorFlow backend.


In [2]:
# Loading variables
# Get train set
X_train = pickle.load(open('../input/final-fer/pickle/X_train.pickle','rb'))
y_train_enc = pickle.load(open('../input/final-fer/pickle/y_train_enc.pickle','rb'))

# Get test set
X_test = pickle.load(open('../input/final-fer/pickle/X_test.pickle','rb'))
y_test_enc = pickle.load(open('../input/final-fer/pickle/y_test_enc.pickle','rb'))

# Normalize datasets
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

X_train /= 255.
X_test /= 255.

In [3]:
print('Shape of Training dataset: ', X_train.shape)
print('Shape of Training labels: ', y_train_enc.shape, '\n')

print('Shape of Testing dataset: ', X_test.shape)
print('Shape of Testing labels: ', y_test_enc.shape)

Shape of Training dataset:  (28821, 48, 48, 3)
Shape of Training labels:  (28821, 7) 

Shape of Testing dataset:  (7066, 48, 48, 3)
Shape of Testing labels:  (7066, 7)


### Model Architecture

In [4]:
# Set hyperparameters
BATCH_SIZE = 64
NUM_CLASSES = 7
EPOCHS = 100
INPUT_SHAPE = (48,48,3)

In [5]:
# Model Architecture
model = Sequential()

model.add(Conv2D(64, kernel_size=(3, 3), activation='relu', input_shape=INPUT_SHAPE, data_format='channels_last', kernel_regularizer=l2(0.01)))
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Dropout(0.5))

model.add(Conv2D(128, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(128, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Dropout(0.5))

model.add(Conv2D(256, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(256, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Dropout(0.5))

model.add(Conv2D(512, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(512, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Dropout(0.5))

model.add(Flatten())

model.add(Dense(512, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(NUM_CLASSES, activation='softmax'))

model.summary()

#Compliling the model with adam optimixer and categorical crossentropy loss
model.compile(loss=categorical_crossentropy,
              optimizer=Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-7),
              metrics=['accuracy'])

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 46, 46, 64)        1792      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 46, 46, 64)        36928     
_________________________________________________________________
batch_normalization_1 (Batch (None, 46, 46, 64)        256       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 23, 23, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 23, 23, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 23, 23, 128)       73856     
_________________________________________________________________
batch_normalization_2 (Batch (None, 23, 23, 128)      

In [6]:
# Train the model
history_cnn = model.fit(X_train, y_train_enc,
                    batch_size=BATCH_SIZE,
                    epochs=EPOCHS,
                    verbose=1,
                    shuffle=True)

Epoch 1/100
28821/28821 [==============================] - 24s 827us/step - loss: 2.0119 - accuracy: 0.2148
Epoch 2/100
28821/28821 [==============================] - 19s 665us/step - loss: 1.8546 - accuracy: 0.2442
Epoch 3/100
28821/28821 [==============================] - 19s 661us/step - loss: 1.8213 - accuracy: 0.2551
Epoch 4/100
28821/28821 [==============================] - 19s 661us/step - loss: 1.7406 - accuracy: 0.2995
Epoch 5/100
28821/28821 [==============================] - 19s 661us/step - loss: 1.6318 - accuracy: 0.3498
Epoch 6/100
28821/28821 [==============================] - 19s 661us/step - loss: 1.5352 - accuracy: 0.3912
Epoch 7/100
28821/28821 [==============================] - 19s 661us/step - loss: 1.4812 - accuracy: 0.4193
Epoch 8/100
28821/28821 [==============================] - 19s 661us/step - loss: 1.4290 - accuracy: 0.4453
Epoch 9/100
28821/28821 [==============================] - 19s 660us/step - loss: 1.3949 - accuracy: 0.4643
Epoch 10/100
28821/28821 [==

In [7]:
# Save the model
history_cnn.save('fer_cnn.h5')

AttributeError: 'History' object has no attribute 'save'

### Prediction

In [8]:
# Save predictions in a variable
y_pred = history_cnn.model.predict(X_test)

In [9]:
# Accuracy
truey=[]
predy=[]

yh = y_pred.tolist()
yt = y_test_enc.tolist()
count = 0

for i in range(len(y_test_enc)):
    yy = max(yh[i])
    yyt = max(yt[i])
    predy.append(yh[i].index(yy))
    truey.append(yt[i].index(yyt))
    if(yh[i].index(yy)== yt[i].index(yyt)):
        count+=1
        
acc = (count/len(y_test_enc))*100
np.save('truey', truey)
np.save('predy', predy)
print("Predicted and true label values saved")
print("Accuracy on test set :"+str(acc)+"%")

Predicted and true label values saved
Accuracy on test set :65.6241154825927%
